In [8]:
from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch(['localhost'],http_auth=('elastic', 'changeme'),port=9200)

In [9]:
doc = {
    'author': 'boto',
    'text': 'boto love jazz',
    'label':'negative',
    'timestamp': datetime.now(),
}

In [10]:
res = es.index(index="test-python", doc_type='news', id=3, body=doc)


In [11]:
def query_item(id):
    res = es.get(index="test-python", doc_type='news', id=id)
    #print(res['_source'])
    item=res['_source']
    content=item['text']
    sentiment=item['label']
    return (content,sentiment)

In [14]:
aa=(query_item('2'))
print(aa[0])

Bangkok is metropolitan of Thailand


In [7]:
print(aa[0])
print(aa[1])

Elasticsearch: cool. bonsai cool.
positive


In [71]:
#item=res['_source']

In [72]:
#content=item['text']

In [76]:
#sentiment=item['label']

In [10]:
#result=res['_source']

In [12]:
def token_item(content):
    ef=es.indices.analyze(text=content,analyzer="thai")
    gc=[]
    for ge in ef['tokens']:
        gc.append(ge['token'])
    return gc

In [79]:
fe=ef['tokens']

In [80]:
for ge in ef['tokens']:
    print(ge['token'])

elasticsearch
cool
bonsai
cool


In [81]:
gc=[]
for ge in ef['tokens']:
    gc.append(ge['token'])

In [94]:
ga=(gc,sentiment)

In [95]:
ga

(['elasticsearch', 'cool', 'bonsai', 'cool'], 'positive')

In [15]:
tk=token_item(aa[0])

In [16]:
gg=(tk,aa[1])

In [17]:
gg

(['bangkok', 'is', 'metropolitan', 'of', 'thailand'], 'positive')

In [18]:
res = es.search(index="test-python", body={"query": {"match_all": {}}})
print("Got %d Hits:" % res['hits']['total'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(author)s: %(text)s" % hit["_source"])

Got 3 Hits:
2017-02-16T21:58:24.844689 jimbo: Bangkok is metropolitan of Thailand
2017-02-16T20:49:56.885727 jimbo: Elasticsearch: cool. bonsai cool.
2017-02-17T09:35:07.921608 boto: boto love jazz


In [26]:
res = es.search(index="test-python", body={"query": {"match_all": {}}})
print("Got %d Hits:" % res['hits']['total'])
ox=[]
for hit in res['hits']['hits']:
    #print(hit['_id'])
    #print(hit['_source']['author'])
    aa=(query_item(hit['_id']))
    tk=token_item(aa[0])
    gg=(tk,aa[1])
    ox.append(gg)

print(ox)

Got 3 Hits:
[(['bangkok', 'is', 'metropolitan', 'of', 'thailand'], 'positive'), (['elasticsearch', 'cool', 'bonsai', 'cool'], 'positive'), (['boto', 'love', 'jazz'], 'negative')]


In [27]:
import nltk
from pythainlp.segment import segment
from time import gmtime, strftime
import codecs

In [28]:
def get_words_in_tweets(tweets):
    all_words = []
    for (words, sentiment) in tweets:
      all_words.extend(words)
    return all_words

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [29]:
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [30]:
training_set = nltk.classify.apply_features(extract_features, ox)

In [32]:
word_features = get_word_features(get_words_in_tweets(ox))

In [33]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [34]:
#  ใส่ข้อความตรงอักษรสีแดง ด้านล่างนี้ #

steep= 'I love lampang'

# ระบุ Catagory  : Social ,Technology, Economic, Environment, Polotic #

catatory = "Environment"


steep=steep.replace(",","")
steepx=segment(steep)
myString = " ".join(steepx)

answer=(classifier.classify(extract_features(myString.split())))
print (steep+" : "+classifier.classify(extract_features(myString.split())))

###
#date=strftime("%Y-%m-%d", gmtime())
#text_file = codecs.open("D:/search/logstash-5.0.0/bin/industry4/Output.csv", "a", 'UTF-8')
#text_file.write(date+","+catatory+","+steep+","+str(answer)+"\n")
#text_file.close()
###

I love lampang : negative


In [47]:
def get_max_id(index):
    res = es.search(index=index, 
                            body={  "stored_fields": [ "_uid"],
                                        "query": {"match_all": {}  },
                                        "size": 1,
                                        "sort": [ { "_uid" : { "order" : "desc"} } ]
                                        })
    return res['hits']['hits'][0]["_id"]

In [48]:
print(get_max_id("test-python"))

3
